# Data Cleaning

### Import Relevant Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore", category = DeprecationWarning)
warnings.filterwarnings("ignore", category = FutureWarning)
warnings.filterwarnings("ignore", category = UserWarning)

In [2]:
# CSV with basic draft info
player_info = pd.read_csv('./Data/nfl_draft_prospects.csv')

# CSV with the scouting reports for players
draft = pd.read_csv('./Data/nfl_draft_profiles_update.csv')

# Excel file with the 2022 scouting reports
draft2022 = pd.read_excel('./Data/reports2022.xlsx',index_col=0)

In [3]:
# Combine draft info with scouting reports
scout2 = draft.merge(player_info,how='left',on='player_id')

In [4]:
scout2.columns

Index(['player_id', 'guid_x', 'alt_player_id', 'player_name_x', 'position_x',
       'pos_abbr_x', 'weight_x', 'height_x', 'player_image_x', 'link_x',
       'school_logo', 'school_x', 'school_abbr_x', 'school_name_x', 'pos_rk_x',
       'ovr_rk_x', 'grade_x', 'text1', 'text2', 'text3', 'text4', 'draft_year',
       'player_name_y', 'concat', 'position_y', 'pos_abbr_y', 'school_y',
       'school_name_y', 'school_abbr_y', 'link_y', 'pick', 'overall', 'round',
       'traded', 'trade_note', 'team', 'team_abbr', 'team_logo_espn', 'guid_y',
       'weight_y', 'height_y', 'pos_rk_y', 'ovr_rk_y', 'grade_y',
       'player_image_y'],
      dtype='object')

In [5]:
# Add 2022 draft info and reports to 2005-2021 data
add2022 = scout2.append(draft2022)

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\3422529145.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  add2022 = scout2.append(draft2022)


In [6]:
# Select only relevant columns
scout = add2022[['player_name_x','pos_abbr_x','weight_x','height_x','school_x','pos_rk_x','ovr_rk_x','grade_x','text1','text2','text3','text4','draft_year','overall','round','team_abbr']]

In [7]:
# Find all players that have a report
texts_orig = scout[(scout.text1.notna())|(scout.text2.notna())|(scout.text3.notna())|(scout.text4.notna())]

In [8]:
# Select only players and reports from 2005 or later
texts = scout[scout.draft_year >= 2005]

In [9]:
# Create blank column to put the final report into
texts['report'] = None

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\982957188.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts['report'] = None


### Remove Post-Draft Analysis text from scouting report

In [10]:
texts.text2[(texts.text2.str.contains("How he fits",na=False)) & (texts.text1.isna())] = texts.text2[(texts.text2.str.contains("How he fits",na=False)) & (texts.text1.isna())].apply(lambda x: str(x).split("How he fits")[0])

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\533545784.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.text2[(texts.text2.str.contains("How he fits",na=False)) & (texts.text1.isna())] = texts.text2[(texts.text2.str.contains("How he fits",na=False)) & (texts.text1.isna())].apply(lambda x: str(x).split("How he fits")[0])
C:\Users\capta\AppData\Local\Temp\ipykernel_24356\533545784.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.text2[(texts.text2.str.contains("How he fits",na=False)) & (texts.text1.isna())] = texts.text2[(texts.text2.str.contains("How he fits",na=False)) & (texts.text1.isna())].ap

### Remove "What he brings" header from scouting report

In [11]:
texts.text2[(texts.text2.str.contains("What he brings",na=False)) & (texts.text1.isna())] = texts.text2[(texts.text2.str.contains("What he brings",na=False)) & (texts.text1.isna())].apply(lambda x: str(x).split("What he brings")[1])

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\3585158994.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.text2[(texts.text2.str.contains("What he brings",na=False)) & (texts.text1.isna())] = texts.text2[(texts.text2.str.contains("What he brings",na=False)) & (texts.text1.isna())].apply(lambda x: str(x).split("What he brings")[1])
C:\Users\capta\AppData\Local\Temp\ipykernel_24356\3585158994.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.text2[(texts.text2.str.contains("What he brings",na=False)) & (texts.text1.isna())] = texts.text2[(texts.text2.str.contains("What he brings",na=False)) & (texts.

### Move the best report for each player from the text column into the report column

In [12]:
texts.loc[(texts.text1.notna()),'report'] = texts.loc[(texts.text1.notna())].text1 

In [13]:
texts.loc[(texts.text4.notna())&(texts.text1.isna()),'report'] = texts.loc[(texts.text4.notna())&(texts.text1.isna())].text4

In [14]:
texts.loc[(texts.text2.notna())&(texts.text1.isna()),'report'] = texts.loc[(texts.text2.notna())&(texts.text1.isna())].text2 

In [15]:
texts.loc[(texts.text2.isna())&(texts.text1.isna())&(texts.text3.notna()),'report'] = texts.loc[(texts.text2.isna())&(texts.text1.isna())&(texts.text3.notna())].text3 

In [16]:
texts.reset_index(inplace=True)

In [17]:
texts.drop(['index'],axis=1,inplace=True)

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\3926660189.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.drop(['index'],axis=1,inplace=True)


In [18]:
texts['rawreport'] = texts.report

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\3885968547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts['rawreport'] = texts.report


### Clean report text to be used in NLP

In [19]:
texts.report[texts.report.str.contains('<br>',na=False)] = texts.report[texts.report.str.contains('<br>',na=False)].apply(lambda x: str(x).split('<br>')[0])

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\1627949020.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('<br>',na=False)] = texts.report[texts.report.str.contains('<br>',na=False)].apply(lambda x: str(x).split('<br>')[0])


In [20]:
texts.report[texts.report.str.contains('<p>',na=False)] = texts.report[texts.report.str.contains('<p>',na=False)].apply(lambda x: str(x).replace('<p>',""))

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\3308517834.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('<p>',na=False)] = texts.report[texts.report.str.contains('<p>',na=False)].apply(lambda x: str(x).replace('<p>',""))


In [21]:
texts.report[texts.report.str.contains('</p>',na=False)] = texts.report[texts.report.str.contains('</p>',na=False)].apply(lambda x: str(x).replace('</p>',""))

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\2662233382.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('</p>',na=False)] = texts.report[texts.report.str.contains('</p>',na=False)].apply(lambda x: str(x).replace('</p>',""))


In [22]:
texts.report[texts.report.str.contains('<a',na=False)] = texts.report[texts.report.str.contains('<a',na=False)].apply(lambda x: str(x).split('<a')[0])

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\2465023052.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('<a',na=False)] = texts.report[texts.report.str.contains('<a',na=False)].apply(lambda x: str(x).split('<a')[0])


In [23]:
texts.report[texts.report.str.contains('Video analysis',na=False)] = texts.report[texts.report.str.contains('Video analysis',na=False)].apply(lambda x: str(x).split('Video analysis')[0])

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\1723811321.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('Video analysis',na=False)] = texts.report[texts.report.str.contains('Video analysis',na=False)].apply(lambda x: str(x).split('Video analysis')[0])


In [24]:
texts.report[texts.report.str.contains('</br>\r\n',na=False)] = texts.report[texts.report.str.contains('</br>\r\n',na=False)].apply(lambda x: str(x).replace('</br>\r\n',""))

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\2806286090.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('</br>\r\n',na=False)] = texts.report[texts.report.str.contains('</br>\r\n',na=False)].apply(lambda x: str(x).replace('</br>\r\n',""))


In [25]:
texts.report[texts.report.str.contains('<em>',na=False)] = texts.report[texts.report.str.contains('<em>',na=False)].apply(lambda x: str(x).split('<em>')[0])

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\512464433.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('<em>',na=False)] = texts.report[texts.report.str.contains('<em>',na=False)].apply(lambda x: str(x).split('<em>')[0])


In [26]:
texts.report[texts.report.str.contains('<i>--',na=False)] = texts.report[texts.report.str.contains('<i>--',na=False)].apply(lambda x: str(x).split('<i>--')[0])

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\2362230250.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('<i>--',na=False)] = texts.report[texts.report.str.contains('<i>--',na=False)].apply(lambda x: str(x).split('<i>--')[0])


In [27]:
texts.report[texts.report.str.contains('<i> -',na=False)] = texts.report[texts.report.str.contains('<i> -',na=False)].apply(lambda x: str(x).split('<i> -')[0])

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\1846342459.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('<i> -',na=False)] = texts.report[texts.report.str.contains('<i> -',na=False)].apply(lambda x: str(x).split('<i> -')[0])


In [28]:
texts.report[texts.report.str.contains('<i>',na=False)] = texts.report[texts.report.str.contains('<i>',na=False)].apply(lambda x: str(x).replace('<i>',""))

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\1689045211.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('<i>',na=False)] = texts.report[texts.report.str.contains('<i>',na=False)].apply(lambda x: str(x).replace('<i>',""))


In [29]:
texts.report[texts.report.str.contains('</i>',na=False)] = texts.report[texts.report.str.contains('</i>',na=False)].apply(lambda x: str(x).replace('</i>',""))

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\1414034982.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('</i>',na=False)] = texts.report[texts.report.str.contains('</i>',na=False)].apply(lambda x: str(x).replace('</i>',""))


In [30]:
texts.report[texts.report.str.contains('</strong> ',na=False)] = texts.report[texts.report.str.contains('</strong> ',na=False)].apply(lambda x: str(x).replace('</strong> ',""))

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\2463272057.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('</strong> ',na=False)] = texts.report[texts.report.str.contains('</strong> ',na=False)].apply(lambda x: str(x).replace('</strong> ',""))


In [31]:
texts.report[texts.report.str.contains('\r\n<strong>',na=False)] = texts.report[texts.report.str.contains('\r\n<strong>',na=False)].apply(lambda x: str(x).replace('\r\n<strong>',""))

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\3622998381.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('\r\n<strong>',na=False)] = texts.report[texts.report.str.contains('\r\n<strong>',na=False)].apply(lambda x: str(x).replace('\r\n<strong>',""))


In [32]:
texts.report[texts.report.str.contains('</strong>\r\n',na=False)] = texts.report[texts.report.str.contains('</strong>\r\n',na=False)].apply(lambda x: str(x).replace('</strong>\r\n',""))

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\2152031530.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('</strong>\r\n',na=False)] = texts.report[texts.report.str.contains('</strong>\r\n',na=False)].apply(lambda x: str(x).replace('</strong>\r\n',""))


In [33]:
texts.report[texts.report.str.contains('<strong>',na=False)] = texts.report[texts.report.str.contains('<strong>',na=False)].apply(lambda x: str(x).replace('<strong>',""))

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\3760746426.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('<strong>',na=False)] = texts.report[texts.report.str.contains('<strong>',na=False)].apply(lambda x: str(x).replace('<strong>',""))


In [34]:
texts.report[texts.report.str.contains('</p',na=False)] = texts.report[texts.report.str.contains('</p',na=False)].apply(lambda x: str(x).replace('</p',""))

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\3611240546.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  texts.report[texts.report.str.contains('</p',na=False)] = texts.report[texts.report.str.contains('</p',na=False)].apply(lambda x: str(x).replace('</p',""))


In [35]:
texts.columns

Index(['player_name_x', 'pos_abbr_x', 'weight_x', 'height_x', 'school_x',
       'pos_rk_x', 'ovr_rk_x', 'grade_x', 'text1', 'text2', 'text3', 'text4',
       'draft_year', 'overall', 'round', 'team_abbr', 'report', 'rawreport'],
      dtype='object')

In [36]:
# Select only relevant columns for NLP
report = texts[['player_name_x','report','round','draft_year','overall','pos_abbr_x','weight_x','height_x','pos_rk_x','ovr_rk_x','grade_x','school_x']]

In [37]:
# Rename columns
report.columns=['player','report','Round','year','pick','pos','weight','height','pos_rk','ovr_rk','grade','school']

### Fill missing Round, Pick, and Pos information

In [39]:
reportnon22 = report[report.year != 2022]

In [40]:
report22 = report[report.year == 2022]

In [42]:
# All drafted players from 2005 - 2022
drafts0522 = pd.read_parquet('./Data/draft0522.parquet')

In [43]:
drafts0522

,Draft_Yr,Rnd,Pick,Tm,Player,Pos,Age,To,College/Univ,Player_NFL_Link,Player_NCAA_Link
0,2005,1,1,SFO,Alex Smith,QB,21,2020,Utah,/players/S/SmitAl03.htm,http://www.sports-reference.com/cfb/players/al...
1,2005,1,2,MIA,Ronnie Brown,RB,23,2014,Auburn,/players/B/BrowRo05.htm,http://www.sports-reference.com/cfb/players/ro...
2,2005,1,3,CLE,Braylon Edwards,WR,22,2012,Michigan,/players/E/EdwaBr00.htm,http://www.sports-reference.com/cfb/players/br...
3,2005,1,4,CHI,Cedric Benson,RB,22,2012,Texas,/players/B/BensCe00.htm,http://www.sports-reference.com/cfb/players/ce...
4,2005,1,5,TAM,Cadillac Williams,RB,23,2011,Auburn,/players/W/WillCa02.htm,http://www.sports-reference.com/cfb/players/ca...
...,...,...,...,...,...,...,...,...,...,...,...
4696,2022,7,258,GNB,Samori Toure,WR,24,2022,Nebraska,/players/T/TourSa00.htm,http://www.sports-reference.com/cfb/players/sa...
4697,2022,7,259,KAN,Nazeeh Johnson,SAF,24,2022,Marshall,/players/J/JohnNa01.htm,http://www.sports-reference.com/cfb/players/na...
4698,2022,7,260,LAC,Zander Horvath,RB,23,2022,Purdue,/players/H/HorvZa00.htm,http://www.sports-reference.com/cfb/players/al...
4699,2022,7,261,LAR,AJ Arcuri,OT,25,2022,Michigan St.,/players/A/ArcuAJ00.htm,http://www.sports-reference.com/cfb/players/aj...


In [44]:
# Update positions
drafts0522.Pos[drafts0522.Pos == 'NT'] = 'DT'
drafts0522.Pos[drafts0522.Pos == 'SAF'] = 'S'

#df.Pos[(df.Pos == 'OLB')|(df.Pos == 'ILB')] = 'LB'
#df.Pos[(df.Pos == 'T')|(df.Pos == 'OT')|(df.Pos == 'C')|(df.Pos == 'G')] = 'OL'

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\2733673970.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts0522.Pos[drafts0522.Pos == 'NT'] = 'DT'
C:\Users\capta\AppData\Local\Temp\ipykernel_24356\2733673970.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts0522.Pos[drafts0522.Pos == 'SAF'] = 'S'


In [45]:
drafts0522.columns = ['year','Round','pick','tm','player','pos','age','to','school','link','slink']

### Clean up player names from draft info to match with scouting report info

In [48]:
drafts0522['player'] = drafts0522.player.apply(lambda x: str(x).strip())
drafts0522.player[(drafts0522.player.str.contains('II'))] = drafts0522.player[(drafts0522.player.str.contains('II'))].apply(lambda x: str(x).split(' II')[0])
drafts0522.player[(drafts0522.player.str.contains('Jr.'))] = drafts0522.player[(drafts0522.player.str.contains('Jr.'))].apply(lambda x: str(x).split(' Jr.')[0])
drafts0522['namelen'] =drafts0522['player'].str.count('\s+')

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\3557052013.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts0522.player[(drafts0522.player.str.contains('II'))] = drafts0522.player[(drafts0522.player.str.contains('II'))].apply(lambda x: str(x).split(' II')[0])
C:\Users\capta\AppData\Local\Temp\ipykernel_24356\3557052013.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts0522.player[(drafts0522.player.str.contains('Jr.'))] = drafts0522.player[(drafts0522.player.str.contains('Jr.'))].apply(lambda x: str(x).split(' Jr.')[0])


In [49]:
# Merge scouting reports with draft info
merged = report.merge(drafts0522,how='left',on=['player','year'])

In [51]:
# Only select players who have a pick number (the target)
merged_final = merged[(merged.pick_x.notna()) | (merged.pick_y.notna())]

In [52]:
merged_final.columns

Index(['player', 'report', 'Round_x', 'year', 'pick_x', 'pos_x', 'weight',
       'height', 'pos_rk', 'ovr_rk', 'grade', 'school_x', 'Round_y', 'pick_y',
       'tm', 'pos_y', 'age', 'to', 'school_y', 'link', 'slink', 'namelen'],
      dtype='object')

In [53]:
# Remove unnecessary columns
merged_final.drop(['tm','to','link','slink','namelen'],axis=1,inplace=True)

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\1152508635.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_final.drop(['tm','to','link','slink','namelen'],axis=1,inplace=True)


### Clean up bad merges on players with the same name in the same draft

In [54]:
merged_final[merged_final.player == 'Alex Smith']

,player,report,Round_x,year,pick_x,pos_x,weight,height,pos_rk,ovr_rk,grade,school_x,Round_y,pick_y,pos_y,age,school_y
0,Alex Smith,Nephew of Michigan State head coach John L. Sm...,1.0,2005.0,1.0,QB,217.0,76.125,2.0,3.0,98.0,Utah,1,1,QB,21,Utah
1,Alex Smith,Nephew of Michigan State head coach John L. Sm...,1.0,2005.0,1.0,QB,217.0,76.125,2.0,3.0,98.0,Utah,3,71,TE,23,Stanford
70,Alex Smith,"Smith redshirted in 2000, played as a backup i...",3.0,2005.0,71.0,TE,258.0,76.125,2.0,60.0,83.0,Stanford,1,1,QB,21,Utah
71,Alex Smith,"Smith redshirted in 2000, played as a backup i...",3.0,2005.0,71.0,TE,258.0,76.125,2.0,60.0,83.0,Stanford,3,71,TE,23,Stanford


In [55]:
merged_final.drop(index=[1,70],inplace=True)

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\2043428584.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_final.drop(index=[1,70],inplace=True)


In [58]:
merged_final.columns

Index(['player', 'report', 'Round_x', 'year', 'pick_x', 'pos_x', 'weight',
       'height', 'pos_rk', 'ovr_rk', 'grade', 'school_x', 'Round_y', 'pick_y',
       'pos_y', 'age', 'school_y'],
      dtype='object')

In [60]:
# Create new columns to bring in the all the round and pick info
merged_final['Round'] = None
merged_final['Pick'] = None

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\516530601.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_final['Round'] = None
C:\Users\capta\AppData\Local\Temp\ipykernel_24356\516530601.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_final['Pick'] = None


In [67]:
merged_final[['Round_y','pick_y']] = merged_final[['Round_y','pick_y']].astype('Int64')

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\2403453810.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_final[['Round_y','pick_y']] = merged_final[['Round_y','pick_y']].astype('Int64')


In [113]:
merged_final[merged_final.player == 'Michael Carter']

,player,report,Round_x,year,pick_x,pos_x,weight,height,pos_rk,ovr_rk,grade,school_x,Round_y,pick_y,pos_y,age,school_y,Round,Pick
5261,Michael Carter,Carter is an undersized scat-back prospect wit...,4.0,2021.0,107.0,RB,201.0,67.875,5.0,83.0,74.0,North Carolina,4,107,RB,22,North Carolina,None,None
5262,Michael Carter,None,NaN,2021.0,NaN,None,NaN,NaN,NaN,NaN,NaN,None,5,154,S,22,Duke,None,None


In [84]:
merged_final.loc[[201],['report','weight','height','pos_rk','ovr_rk','grade']] = None
merged_final.loc[[200],['Round_x','pick_x','pos_x']] = None

In [92]:
merged_final.loc[[2243],['report','weight','height','pos_rk','ovr_rk','grade','Round_x','pick_x','pos_x']] = None

In [102]:
merged_final.loc[[3531],['Round_y','pick_y','school_y','age']] = np.nan

In [108]:
merged_final.loc[[4507],['report','weight','height','pos_rk','ovr_rk','grade','Round_x','pick_x','pos_x','school_x']] = None

In [112]:
merged_final.loc[[5262],['report','weight','height','pos_rk','ovr_rk','grade','Round_x','pick_x','pos_x','school_x']] = None

In [88]:
merged_final.drop(index=[15,16],inplace=True)

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\2619285946.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_final.drop(index=[15,16],inplace=True)


In [114]:
merged_final[(merged_final.Round_x != merged_final.Round_y) & (merged_final.Round_x.notna())]

,player,report,Round_x,year,pick_x,pos_x,weight,height,pos_rk,ovr_rk,grade,school_x,Round_y,pick_y,pos_y,age,school_y,Round,Pick


In [130]:
# Set final Round and pick
merged_final.Round[merged_final.Round_x.notna()] = merged_final.Round_x[merged_final.Round_x.notna()]
merged_final.Pick[merged_final.pick_x.notna()] = merged_final.pick_x[merged_final.pick_x.notna()]
merged_final.Round[merged_final.Round_y.notna()] = merged_final.Round_y[merged_final.Round_y.notna()]
merged_final.Pick[merged_final.pick_y.notna()] = merged_final.pick_y[merged_final.pick_y.notna()]

C:\Users\capta\AppData\Local\Temp\ipykernel_24356\377890902.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_final.Round[merged_final.Round_x.notna()] = merged_final.Round_x[merged_final.Round_x.notna()]
C:\Users\capta\AppData\Local\Temp\ipykernel_24356\377890902.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_final.Round[merged_final.Round_x.notna()] = merged_final.Round_x[merged_final.Round_x.notna()]
C:\Users\capta\AppData\Local\Temp\ipykernel_24356\377890902.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: ht

In [134]:
merged_final[merged_final.report.isna()]

,player,report,Round_x,year,pick_x,pos_x,weight,height,pos_rk,ovr_rk,grade,school_x,Round_y,pick_y,pos_y,age,school_y,Round,Pick
124,Todd Herremans,None,4.0,2005.0,126.0,OT,303.0,78.125,41.0,NaN,30.0,Saginaw Valley,4,126,T,22,Saginaw Valley St.,4,126
133,Matt Giordano,None,4.0,2005.0,135.0,DB,195.0,70.625,45.0,NaN,30.0,California,4,135,DB,22,California,4,135
155,Jeb Huckeba,None,5.0,2005.0,159.0,DE,247.0,76.375,37.0,NaN,30.0,Arkansas,5,159,LB,,Arkansas,5,159
159,Frank Omiyale,None,5.0,2005.0,163.0,OT,310.0,76.250,32.0,NaN,30.0,Tennessee Tech,5,163,T,22,Tennessee Tech,5,163
163,Michael Hawkins,None,5.0,2005.0,167.0,DB,0.0,0.000,NaN,NaN,NaN,Oklahoma,5,167,DB,22,Oklahoma,5,167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3270,A.J. Derby,None,NaN,2015.0,NaN,TE,255.0,75.750,16.0,263.0,38.0,Arkansas,6,202,TE,23,Arkansas,6,202
3325,Joey Iosefa,None,7.0,2015.0,231.0,FB,247.0,71.750,4.0,334.0,31.0,Hawai'i,7,231,FB,24,Hawaii,7,231
4507,David Long,None,NaN,2019.0,NaN,None,NaN,NaN,NaN,NaN,NaN,None,6,188,LB,22,West Virginia,6,188
4627,Gerri Green,None,6.0,2019.0,199.0,OLB,252.0,76.000,29.0,336.0,34.0,Mississippi State,6,199,DE,23,Mississippi St.,6,199


In [136]:
# Remove unnecessary columns for the final dataframe
final = merged_final.drop(['Round_x','pick_x','school_x','Round_y','pick_y','school_y'],axis=1)

In [139]:
final[['Round','Pick']] = final[['Round','Pick']].astype('Int64')

In [141]:
# Only select rows with a report
final = final[final.report.notna()]
final

,player,report,year,pos_x,weight,height,pos_rk,ovr_rk,grade,pos_y,age,Round,Pick
0,Alex Smith,Nephew of Michigan State head coach John L. Sm...,2005.0,QB,217.0,76.125,2.0,3.0,98.0,QB,21,1,1
2,Ronnie Brown,Brown has played second fiddle to Carnell Will...,2005.0,RB,233.0,72.250,1.0,4.0,98.0,RB,23,1,2
3,Braylon Edwards,"Edwards' father, Stanley, played at Michigan a...",2005.0,WR,211.0,74.875,1.0,1.0,99.0,WR,22,1,3
4,Cedric Benson,Benson was drafted by the Los Angeles Dodgers ...,2005.0,RB,222.0,70.500,3.0,10.0,96.0,RB,22,1,4
5,Carnell Williams,Williams started two games and played in nine ...,2005.0,RB,217.0,70.875,2.0,6.0,97.0,NaN,NaN,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5740,Jeff Gunter,Gunter is a versatile edge defender who lines ...,2022.0,NaN,NaN,NaN,NaN,NaN,NaN,OLB,23,7,252
5741,Trenton Gill,Gill has a strong leg. He led the ACC in punt ...,2022.0,NaN,NaN,NaN,NaN,NaN,NaN,P,23,7,255
5742,Jesse Luketa,Luketa is a relentless run-defender who has go...,2022.0,NaN,NaN,NaN,NaN,NaN,NaN,LB,23,7,256
5743,Marquis Hayes,Hayes is a mauler who plays with good leverage...,2022.0,NaN,NaN,NaN,NaN,NaN,NaN,OL,23,7,257


### Save report dataframe

In [142]:
final.to_parquet('./Data/draft_reports.parquet')